In [1]:
import requests
import pandas as pd
from bokeh.io import output_file, show
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, FactorRange,CustomJS, RadioGroup,Row

url = 'https://camparator.com/compare/cam/'
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[-1]
# print(df)
df.to_csv('../data/CAMS_DATA.csv')


ImportError: lxml not found, please install it

In [5]:
data=pd.read_csv('../data/CAMS_DATA.csv')
data.drop(columns=['Name','Unnamed: 0', 'Unnamed: 0.1','Unnamed: 14'],inplace=True)
data.drop(data.loc[data['Generation']!='Current'].index,inplace=True)
data.drop(columns=['Generation'],inplace=True)
data.reset_index(drop=True,inplace=True)
data =data.rename(columns = {'Lower (mm)':'Lower','Upper (mm)':'Upper','Range (mm)':'Range'})
data.to_csv('../data/CAMS.csv')

#drop cams that are not interesting for me
data = data.drop(data.loc[data['Brand']=='Valley Giant',:].index)
data = data.drop(data.loc[data['Brand']=='Fixe Hardware',:].index)
data = data.drop(data.loc[data['Brand']=='Trango',:].index)

In [94]:
def function(data):
    return data.Weight.sum()
data_weight = data.groupby(['Brand','Entity']).apply(function)


In [103]:
output_file("../plots/CAMSWEIGHT_joined.html")

y = [(brand, model,size) for brand,model,size in zip(data.Brand.values,data.Entity.values,data.Size.values)]

source = ColumnDataSource(data = dict(y=y,Lower=data.Lower,Upper=data.Upper,weight=data.Weight,
                                      set_weight = data.SetWeight))
p_range = figure(y_range=FactorRange(*y), plot_height=650,plot_width=600,
           title="CAMS COMPARASION")
p_range.hbar(y='y',left='Lower',right='Upper',height=0.4,source=source)




p_weight = figure(y_range=FactorRange(*x), plot_height=650,plot_width=600,
           title="CAMS COMPARASION")
p_weight.hbar(y='y', right='weight', height=0.4, source=source)

p_weight.circle(y=data_weight.index,x=data_weight.values,color='red')


show(Row(p_range,p_weight))